## Auto_ML

### 배경

    - 데이터 타입에 상관없이 특정 데이터 입력 시 자동으로 모델을 선정하고 성능을 평가하여 결과를 제시해 주는 autoML 역량 내재화를 목적으로 합니다.

### 사용 가능 데이터

    - 우선, table data에 대한 분석만 가능하도록 만들었으며, 추후 다양한 데이터에 대해 적용이 가능하도록 업데이트할 예정입니다.
    
    - Pycaret, autokeras, Tabnet 모델을 활용하며, 해당 모델을 구성하였습니다.
    

### 모델 사용 환경 및 세팅

    DataFrame Input

    target 변수는 마지막 column에 배치

    scikit-learn version 0.23.2

    tensorflow 2.5

    python 3.8 사용

    pycaret, autokeras
    
### 데이터

    - 데이터는 크게 회귀를 위한 것과 분류를 위한 것으로 나뉜다.
    
    분류 과제에서는 scikit-laern 의 iris data를 활용해 분석을 진행했다.
    
    회귀 과제에서는 skckit-learn 의 boston data를 활용해 분석을 진행했다.
    
### 실험 세팅

    - 모든 데이터에서 20 % 는 test data 로 사용하며 학습 과정에 전혀 사용하지 않고 마지막 모델들을 비교할 때만 사용하였다.
    
    - 각 학습 과정에서는 train data 를 train 과 valdiation으로 나누어 학습을 진행했다.
    
### 결과 평가

    - 분류 모델에서는 Accuracy 로, 회귀 모델에서는 Mean absolute error 로 모델을 성능을 비교했다.

In [1]:
# import
from tensorflow import keras
import pycaret
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import tensorflow as tf
import autokeras as ak
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, confusion_matrix
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

In [2]:
from pycaret.classification import *

In [3]:
class classification_model():
    
    
    def __init__(self, df):
        self.df = df
        
        
    def preprocess(self):
        target_name = self.df.columns[-1]
        
        
        # 학습에 사용될 dataset에 대한 transforamtion 진행 / 이 중 20% 를 test set 으로 사용하며 이는 학습에 사용되지 않음
        self.exp_clf = setup(data = self.df, target = target_name, transformation = True, normalize = True,
                       session_id=123, log_experiment=True, experiment_name='classification', fold_shuffle=True,
                       imputation_type='iterative', train_size = 0.8)
        
        for i in self.exp_clf:
            if isinstance(i, list):
                if len(i) > 4:
                    if i[0][0]=='Setup Config':
                        #train / test size 를 출력
                        self.X_train = i[1][1]
                        self.y_train = i[2][1]
                        self.X_test = i[3][1]
                        self.y_test = i[4][1]
                        print('X_train_shape : ', self.X_train.shape)
                        print('y_train_shape : ', self.y_train.shape)
                        print('X_test_shape : ', self.X_test.shape)
                        print('y_test_shape : ', self.y_test.shape)
        
        
        
    def pycaret_compare_models(self):
        '''
        pycaret 오픈 소스를 통한 모델 생성 및 비교
        
        accuracy 기준으로 10개 fold를 이용해 cross validation 성능이 가장 좋은 top 3 모델을 선정
        
        총 5개의 후보에 대해 비교 진행
        1. 가장 성능이 좋은 것으로 나타난 머신러닝 모델
        2. top 3 모델을 blending한 모델 (hard)
        3. top 3 모델을 blending한 모델 (hard X)
        4. top 3 모델을 stacking한 모델
        5. top 3 모델을 xgboost를 이용해 stacking한 모델
        
        총 5가지 모델 중 accuracy가 가장 좋은 모델을 선정 및 결과 도출
        '''
        exp_clf = self.exp_clf
        top3 = compare_models(n_select=3)
        
        # 가장 성능이 좋은 모델 tuning
        m1 = create_model(top3[0])
        #tuned_model1 = tune_model(m1)
        
        m2 = create_model(top3[1])
        m3 = create_model(top3[2])
        
        # 가장 성능이 좋은 3개의 모델 blend
        #blend_hard = blend_models(estimator_list = [m1, m2, m3], method='hard')
        #blender_top3 = blend_models(top3)
        
        # 가장 성능이 좋은 3개의 모델 stack
        #stack_soft = stack_models(top3)
        
        #xgboost = create_model('xgboost')
        #stack_soft2 = stack_models(top3, meta_model=xgboost)
        
        # 모든 모델 중 하나의 모델 선택
        best_model = automl(use_holdout=True)
        pred_holdout = predict_model(best_model)
        
        return pred_holdout
    
    
    def autokeras_model(self):
        self.y_train = self.y_train.astype('str')
        clf = ak.StructuredDataClassifier(overwrite=True, max_trials=5)
        clf.fit(self.X_train, self.y_train, epochs=50)
        predicted_y = clf.predict(self.X_test)
        
        return predicted_y
    
    
    def Tabnet_model(self):
        nunique = self.X_train.nunique()
        types = self.X_train.dtypes

        categorical_columns = []
        categorical_dims =  {}
        for col in self.X_train.columns:
            if types[col] == 'object' or nunique[col] < len(col)//4:
                # print(col, df_iris[col].nunique())
                l_enc = LabelEncoder()
                self.X_train[col] = self.X_train[col].fillna("VV_likely")
                self.X_train[col] = l_enc.fit_transform(self.X_train[col].values)
                self.X_test[col] = self.X_test[col].fillna("VV_likely")
                self.X_test[col] = l_enc.fit_transform(self.X_tset[col].values)
                categorical_columns.append(col)
                categorical_dims[col] = len(l_enc.classes_)
            else:
                self.X_train.fillna(self.X_train[col].mean(), inplace=True)
                self.X_test.fillna(self.X_test[col].mean(), inplace=True)


        # Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.

        features = [ col for col in self.X_train.columns] 
        cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
        cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

        X_train, X_val, y_train, y_val = train_test_split(self.X_train, self.y_train, test_size = 0.2, random_state=123)

        y_train = y_train.values
        X_train = X_train.values

        y_val = y_val.values
        X_val = X_val.values
        
        X_test = self.X_test.values
        
        clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=10,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-2),
                       scheduler_params={"step_size":50,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='sparsemax' # "sparsemax", entmax
                      )
        
        max_epochs = 30

        clf.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_name=['train', 'val'],
            eval_metric=['accuracy'],
            max_epochs=max_epochs , patience=20,
            batch_size=1024, virtual_batch_size=128,
            num_workers=0,
            weights=1,
            drop_last=False,
        )
        
        preds = clf.predict(X_test)
        
        return preds
    
    def forward(self):
        self.preprocess()
        print("Model Training Start!")
        '''
        각 모델들을 학습시키고 결과 비교 시작
        
        '''
        pycaret_ = self.pycaret_compare_models()
        autokeras_ = self.autokeras_model()
        tabnet_ = self.Tabnet_model()
        
        pycaret_acc = accuracy_score(pycaret_['Label'], self.y_test)
        y_test = self.y_test.astype('str')
        autokeras_acc = accuracy_score(autokeras_, y_test)
        y_test_ = self.y_test.values
        tabnet_acc = accuracy_score(tabnet_, y_test_)
        
        print("Accuracy for each autoML models result")
        print("Pycaret: ", pycaret_acc)
        print("Autokeras: ", autokeras_acc)
        print("Tabnet: ", tabnet_acc)

In [4]:
iris = load_iris()
iris_data = np.concatenate([iris.data, iris.target.reshape(-1,1)], 1)
df_iris = pd.DataFrame(data=iris_data, columns = iris.feature_names + ['target'])

In [ ]:
clm = classification_model(df_iris)
clm.forward()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9167,1.0000,0.9167,0.9333,0.9153,0.8750,0.8843
1,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9167,0.9792,0.9167,0.9333,0.9153,0.8750,0.8843
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9167,1.0000,0.9333,0.9333,0.9167,0.8737,0.8830
8,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
9,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000


In [ ]:
from pycaret.regression import *

In [ ]:
class Regression_model():
    
    def __init__(self, df):
        self.df = df
        
        
    def preprocess(self):
        target_name = self.df.columns[-1]
        self.exp_reg102 = setup(data = self.df, target = target_name, session_id=123,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95,
                  log_experiment = True, experiment_name = 'regression')
        
        for i in self.exp_reg102:
            if isinstance(i, list):
                if len(i) > 4:
                    if i[0][0]=='Setup Config':
                        #train / test size 를 출력
                        self.X_train = i[1][1]
                        self.y_train = i[2][1]
                        self.X_test = i[3][1]
                        self.y_test = i[4][1]
                        print('X_train_shape : ', self.X_train.shape)
                        print('y_train_shape : ', self.y_train.shape)
                        print('X_test_shape : ', self.X_test.shape)
                        print('y_test_shape : ', self.y_test.shape)
    
    
    def pycaret_compare_models(self):
        '''
        pycaret 오픈 소스를 통한 모델 생성 및 비교
        
        accuracy 기준으로 10개 fold를 이용해 cross validation 성능이 가장 좋은 top 3 모델을 선정
        
        총 5개의 후보에 대해 비교 진행
        1. 가장 성능이 좋은 것으로 나타난 머신러닝 모델
        2. top 3 모델을 blending한 모델 (hard)
        3. top 3 모델을 blending한 모델 (hard X)
        4. top 3 모델을 stacking한 모델
        5. top 3 모델을 xgboost를 이용해 stacking한 모델
        
        총 5가지 모델 중 accuracy가 가장 좋은 모델을 선정 및 결과 도출
        '''
        exp_reg102 = self.exp_reg102
        top3 = compare_models(n_select=3)
        
        # 가장 성능이 좋은 모델 tuning
        m1 = create_model(top3[0])
        tuned_m1 = tune_model(m1, n_iter=50)
        
        m2 = create_model(top3[1])
        m3 = create_model(top3[2])
        
        # 가장 성능이 좋은 3개의 모델 blend
        #blend_hard = blend_models(estimator_list = [m1, m2, m3])
        #blender_top3 = blend_models(top3)
        
        # 가장 성능이 좋은 3개의 모델 stack
        #stack_soft = stack_models(top3)
        
        #xgboost = create_model('xgboost')
        #stack_soft2 = stack_models(top3, meta_model=xgboost)
        
        # 모든 모델 중 하나의 모델 선택
        best_model = automl(use_holdout=True)
        pred_holdout = predict_model(best_model)
        
        return pred_holdout

        
    def autokeras_model(self):
        y_train = np.array(self.y_train)
        clf = ak.StructuredDataRegressor(overwrite=True, max_trials=5)
        clf.fit(self.X_train, self.y_train, epochs=50)
        predicted_y = clf.predict(self.X_test)
        
        return predicted_y
   
    def Tabnet_model(self):
        nunique = self.X_train.nunique()
        types = self.X_train.dtypes

        categorical_columns = []
        categorical_dims =  {}
        for col in self.X_train.columns:
            if types[col] == 'object' or nunique[col] < len(col)//4:
                # print(col, df_iris[col].nunique())
                l_enc = LabelEncoder()
                self.X_train[col] = self.X_train[col].fillna("VV_likely")
                self.X_train[col] = l_enc.fit_transform(self.X_train[col].values)
                self.X_test[col] = self.X_test[col].fillna("VV_likely")
                self.X_test[col] = l_enc.fit_transform(self.X_tset[col].values)
                categorical_columns.append(col)
                categorical_dims[col] = len(l_enc.classes_)
            else:
                self.X_train.fillna(self.X_train[col].mean(), inplace=True)
                self.X_test.fillna(self.X_test[col].mean(), inplace=True)


        # Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.

        features = [ col for col in self.X_train.columns] 
        cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
        cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

        X_train, X_val, y_train, y_val = train_test_split(self.X_train, self.y_train, test_size = 0.2, random_state=123)

        y_train = y_train.values.reshape(-1,1)
        X_train = np.array(X_train.values)

        y_val = y_val.values.reshape(-1,1)
        X_val = np.array(X_val.values)
        
        X_test = np.array(self.X_test.values)
        
        clf = TabNetRegressor(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=10,
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-2),
                       scheduler_params={"step_size":50,
                                         "gamma":0.9},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR,
                       mask_type='sparsemax' # "sparsemax", entmax
                      )
        
        max_epochs = 30

        clf.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            eval_name=['train', 'val'],
            eval_metric=['mae'],
            max_epochs=max_epochs , patience=20,
            batch_size=1024, virtual_batch_size=128,
            num_workers=0,
            drop_last=False,
        )
        
        preds = clf.predict(X_test)
        
        return preds
        
    
    def forward(self):
        print("Preprocessing Start")
        self.preprocess()
        print("Preprocessing Done!")
        print("Model Training Start!")
        '''
        각 모델들을 학습시키고 결과 비교 시작
        '''
        pycaret_ = self.pycaret_compare_models()
        autokeras_ = self.autokeras_model()
        tabnet_ = self.Tabnet_model()
        
        pycaret_mae = mean_absolute_error(pycaret_['Label'], self.y_test)
        y_test = self.y_test
        autokeras_mae = mean_absolute_error(autokeras_, y_test)
        tabnet_mae = mean_absolute_error(tabnet_, y_test)
        
        print("Mean absolute error for each autoML models result")
        print("Pycaret: ", pycaret_mae)
        print("Autokeras: ", autokeras_mae)
        print("Tabnet: ", tabnet_mae)
    

In [ ]:
boston = load_boston()
boston_data = np.concatenate([boston.data, boston.target.reshape(-1,1)], 1)
df_boston = pd.DataFrame(boston_data, columns = list(boston.feature_names) + ['target'])

In [ ]:
rg = Regression_model(df_boston)
rg.forward()